# Convert HEALPix Zarr data to netCDF with variable subsets for TempestExtremes

## Adapted from author:
- Zhe Feng || zhe.feng@pnnl.gov

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy import interpolate
import numpy as np

import easygems.healpix as egh
import intake
import uxarray as ux


plt.rcParams['figure.dpi'] = 72

### Load the catalog

In [2]:
list(intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml"))

['CN', 'EU', 'JAPAN', 'NCAR', 'NERSC', 'UK', 'online']

In [3]:
# Load the NERSC catalog
current_location = "NCAR"
cat = intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml")[current_location]
list(cat)

['CERES_EBAF',
 'ERA5',
 'IR_IMERG',
 'JRA3Q',
 'MERRA2',
 'casesm2_10km_nocumulus',
 'icon_d3hp003',
 'icon_d3hp003aug',
 'icon_d3hp003feb',
 'icon_ngc4008',
 'ifs_tco3999-ng5_deepoff',
 'ifs_tco3999-ng5_rcbmf',
 'ifs_tco3999-ng5_rcbmf_cf',
 'mpas_dyamond1',
 'mpas_dyamond2',
 'mpas_dyamond3',
 'nicam_220m_test',
 'nicam_gl11',
 'scream-dkrz',
 'scream2D_hrly',
 'scream_lnd',
 'scream_ne120',
 'tracking-d3hp003',
 'um_Africa_km4p4_RAL3P3_n1280_GAL9_nest',
 'um_CTC_km4p4_RAL3P3_n1280_GAL9_nest',
 'um_SAmer_km4p4_RAL3P3_n1280_GAL9_nest',
 'um_SEA_km4p4_RAL3P3_n1280_GAL9_nest',
 'um_glm_n1280_CoMA9_TBv1p2',
 'um_glm_n1280_GAL9',
 'um_glm_n2560_RAL3p3',
 'wrf_conus',
 'wrf_samerica']

### Pick a Dataset

In [20]:
pd.DataFrame(cat["mpas_dyamond1"].describe()["user_parameters"])

,name,description,type,allowed,default
0,time,temporal resolution of the dataset,str,"[PT15M, PT3H]",PT3H
1,zoom,zoom resolution of the dataset,int,"[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]",7


### Load Data into a Data Set
most datasets have a `zoom` parameter. We will use `zoom` level 8 [(~24km)](https://easy.gems.dkrz.de/Processing/healpix/index.html#healpix-spatial-resolution)

In [24]:
zoom=7
ds= cat["mpas_dyamond1"](zoom=zoom,time='PT15M').to_dask().pipe(egh.attach_coords)
ds

/glade/work/huili7/miniconda3/envs/easy/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
/glade/derecho/scratch/huili7/tmp/ipykernel_67954/3739089240.py:2: UserWarning: No CRS coordinate was found. Attempting to infer it from the dataset shape. Please check the result!
  ds= cat["mpas_dyamond1"](zoom=zoom,time='PT15M').to_dask().pipe(egh.attach_coords)


<xarray.Dataset> Size: 293GB
Dimensions:             (time: 3841, cell: 196608)
Coordinates:
  * time                (time) datetime64[ns] 31kB 2016-08-01 ... 2016-09-10
    crs                 float64 8B nan
  * cell                (cell) int32 786kB 0 1 2 3 ... 196605 196606 196607
    lat                 (cell) float64 2MB 0.2984 0.5968 ... -0.5968 -0.2984
    lon                 (cell) float64 2MB 45.0 45.35 44.65 ... 314.6 315.0
Data variables: (12/97)
    aclwdnb             (time, cell) float32 3GB dask.array<chunksize=(12, 196608), meta=np.ndarray>
    aclwdnt             (time, cell) float32 3GB dask.array<chunksize=(12, 196608), meta=np.ndarray>
    aclwnetb            (time, cell) float32 3GB dask.array<chunksize=(12, 196608), meta=np.ndarray>
    aclwnett            (time, cell) float32 3GB dask.array<chunksize=(12, 196608), meta=np.ndarray>
    aclwupb             (time, cell) float32 3GB dask.array<chunksize=(12, 196608), meta=np.ndarray>
    aclwupt             (time, cell) float32 3GB dask.array<chunksize=(12, 196608), meta=np.ndarray>
    ...                  ...
    w_250hPa            (time, cell) float32 3GB dask.array<chunksize=(12, 196608), meta=np.ndarray>
    w_500hPa            (time, cell) float32 3GB dask.array<chunksize=(12, 196608), meta=np.ndarray>
    w_700hPa            (time, cell) float32 3GB dask.array<chunksize=(12, 196608), meta=np.ndarray>
    w_850hPa            (time, cell) float32 3GB dask.array<chunksize=(12, 196608), meta=np.ndarray>
    w_925hPa            (time, cell) float32 3GB dask.array<chunksize=(12, 196608), meta=np.ndarray>
    zgrid               (time, cell) float32 3GB dask.array<chunksize=(12, 196608), meta=np.ndarray>

In [29]:
ds.zgrid.min().compute()

<xarray.DataArray 'zgrid' ()> Size: 4B
array(nan, dtype=float32)
Coordinates:
    crs      float64 8B nan

In [6]:
zoom = 8
ds30m = cat["mpas_dyamond3"](zoom=zoom,time='PT30M').to_dask().pipe(egh.attach_coords)
uds30m = ux.UxDataset.from_healpix(ds30m)



/glade/work/huili7/miniconda3/envs/easy/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
/glade/derecho/scratch/huili7/tmp/ipykernel_67954/560177439.py:2: UserWarning: No CRS coordinate was found. Attempting to infer it from the dataset shape. Please check the result!
  ds30m = cat["mpas_dyamond3"](zoom=zoom,time='PT30M').to_dask().pipe(egh.attach_coords)
/glade/work/huili7/miniconda3/envs/easy/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:237: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


In [36]:
ds3h = cat["mpas_dyamond3"](zoom=zoom,time='PT3H').to_dask().pipe(egh.attach_coords)
rel100 = ux.UxDataset.from_healpix(ds3h).relhum_100hPa

/glade/work/huili7/miniconda3/envs/easy/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
/glade/derecho/scratch/huili7/tmp/ipykernel_119104/3697034315.py:1: UserWarning: No CRS coordinate was found. Attempting to infer it from the dataset shape. Please check the result!
  ds3h = cat["mpas_dyamond3"](zoom=zoom,time='PT3H').to_dask().pipe(egh.attach_coords)
/glade/work/huili7/miniconda3/envs/easy/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:237: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


In [45]:
uds30m_resample = uds30m.resample(time="3h").mean()
# for dyamond 1, we used instatanous value by resample.first(). It doesn't seem to work with dyamind 3? 

In [54]:
# Variables to output
varout = ['time', 'lat', 'lon', 'umeridional_925hPa', 'umeridional_850hPa','umeridional_200hPa',
          'uzonal_925hPa','uzonal_850hPa','uzonal_200hPa','u10','v10',
         'temperature_850hPa','relhum_850hPa','mslp',
          'height_925hPa','height_850hPa','height_700hPa','height_500hPa','height_250hPa','vorticity_500hPa']
# Subset variables
ds_out = uds30m_resample[varout]


In [55]:
ds_out['relhum_100hPa'] = rel100

### Write to netCDF file

In [60]:
%%time
out_dir = '/glade/derecho/scratch/huili7/mpas_tracking/'
out_filename = f'{out_dir}mpas_dyamond3_hp{zoom}.nc'

# Set encoding options: chunk sizes, compression
encoding = {
    'time': {'contiguous': False},  # Important for unlimited dims
   # 'uivt': {'zlib': True, 'complevel': 1, 'chunksizes': [24, 4**6]},
   # 'vivt': {'zlib': True, 'complevel': 1, 'chunksizes': [24, 4**6]}
}

# # Setting the unlimited dimension takes 16s to write 100 times at zoom=8 (2 variables)
ds_out.to_netcdf(out_filename, format='NETCDF4', unlimited_dims='time', mode='w', encoding=encoding)
print(f"Saved: {out_filename}")

Saved: /glade/derecho/scratch/huili7/mpas_tracking/mpas_dyamond3_hp8.nc
CPU times: user 18min 6s, sys: 1min 39s, total: 19min 46s
Wall time: 5min 48s
